# 🧪 User Acceptance Test (UAT) for Feast Integration

This notebook performs a full end-to-end User Acceptance Test (UAT) for a Feast setup using PostgreSQL for both the offline and online stores, and SQL-based registry.

## ✅ Steps Covered

1. **Load Feast Configuration**  
   Copies the `feature_store.yaml` file from the location specified by the `FEAST_CONFIG_PATH` environment variable into the working directory.

2. **Load Sample Data into Offline Store**  
   Reads `data/driver_stats.parquet` and loads it into the configured PostgreSQL offline store using credentials extracted from `feature_store.yaml`.

3. **Apply Feature Definitions**  
   Cleans up the `specs/` directory, ensures it is a valid Python package, and uses the `FeatureStore.apply()` method to register entities, features, and data sources.

4. **Retrieve Historical Features**  
   Fetches historical training data using `get_historical_features()` to ensure offline retrieval logic is functioning as expected.

5. **Materialize Features to Online Store**  
   Uses `materialize()` or `materialize_incremental()` to populate the online store from the offline store, testing the materialization path.

6. **Retrieve Online Features**  
   Uses `get_online_features()` to ensure real-time retrieval from the online store works correctly with previously materialized data.

---

This notebook verifies that Feast is correctly configured, the stores are accessible and populated, and that both offline and online retrieval workflows function end-to-end.

In [1]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 90.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 115.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uni

# Load Feast Configuration

In [1]:
import os
import shutil
import subprocess
import yaml
from datetime import datetime
from pprint import pprint

import pandas as pd
from sqlalchemy import create_engine, inspect
from feast import FeatureStore

feature_store_yaml_path = os.getenv("FEAST_FS_YAML_FILE_PATH")
with open(feature_store_yaml_path, "r") as f:
    feature_store_file = yaml.safe_load(f)

/opt/conda/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/types/Value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/DataFormat.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/Feature.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/pytho

# Load Sample Data into Offline Store

In [2]:
offline_config = feature_store_file.get("offline_store", {})
db_user = offline_config.get("user")
db_password = offline_config.get("password")
db_host = offline_config.get("host")
db_port = offline_config.get("port")
db_name = offline_config.get("database")

if not all([db_user, db_password, db_host, db_port, db_name]):
    raise ValueError("One or more offline store config values are missing in feature_store.yaml")

parquet_file_path = "data/driver_stats.parquet"
df = pd.read_parquet(parquet_file_path)

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
table_name = "driver_stats"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(
    f"Data from {parquet_file_path} has been loaded into the '{table_name}' table "
    f"in the '{db_name}' database."
)

Data from data/driver_stats.parquet has been loaded into the 'driver_stats' table in the 'offline_store' database.


# Apply Feature Definitions

In [3]:
# Clean up .ipynb_checkpoints to prevent Feast import issues
shutil.rmtree("specs/.ipynb_checkpoints", ignore_errors=True)

# Ensure 'specs' is recognized as a Python module by Feast
with open("specs/__init__.py", "w") as f:
    f.write("")

# Run 'feast apply' from the correct repo directory
specs_dir = os.path.join(os.getcwd(), "specs")

try:
    result = subprocess.run(
        ["feast", "apply"], cwd=specs_dir, capture_output=True, text=True, check=True
    )
    print("Feast apply executed successfully.\n")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("Error running 'feast apply':")
    print(e.stderr)

Feast apply executed successfully.

No project found in the repository. Using project name feast_project defined in feature_store.yaml
Applying changes for project feast_project
Deploying infrastructure for driver_hourly_stats2



In [4]:
registry_uri = feature_store_file.get("registry", {}).get("path")
if not registry_uri:
    raise ValueError("Registry URI not found in feature_store.yaml")

engine = create_engine(registry_uri)
inspector = inspect(engine)
existing_tables = set(inspector.get_table_names())

expected_tables = {
    "projects",
    "entities",
    "data_sources",
    "feature_views",
    "stream_feature_views",
    "on_demand_feature_views",
    "feature_services",
    "saved_datasets",
    "validation_references",
    "managed_infra",
    "permissions",
    "feast_metadata",
}

missing_tables = expected_tables - existing_tables
assert not missing_tables, f"Missing tables in registry DB: {missing_tables}"

print("✅ All expected Feast registry tables are present.")

✅ All expected Feast registry tables are present.


# Retrieve Historical Features

In [5]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        "label_driver_reported_satisfaction": [1, 5, 3],
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

store = FeatureStore(repo_path="specs")

try:
    df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "driver_hourly_stats2:conv_rate",
            "driver_hourly_stats2:acc_rate",
            "driver_hourly_stats2:avg_daily_trips",
        ],
    ).to_df()

    assert not df.empty, "get_historical_features returned an empty DataFrame"
    print("✅ get_historical_features executed successfully and returned data.")
    display(df.head())
except Exception as e:
    print("❌ Error calling get_historical_features:")
    print(str(e))

/opt/conda/lib/python3.11/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(
The `path` of the `RegistryConfig` starts with a plain `postgresql` string. We are updating this to `postgresql+psycopg` to ensure that the `psycopg3` driver is used by `sqlalchemy`. If you want to use `psycopg2` pass `postgresql+psycopg2` explicitely to `path`. To silence this warning, pass `postgresql+psycopg` explicitely to `path`.


✅ get_historical_features executed successfully and returned data.


,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips
0,1001,2021-04-12 10:59:42,1,1,10,0.610259,0.319104,564
1,1002,2021-04-12 08:12:10,5,2,20,0.973000,0.305229,822
2,1003,2021-04-12 16:40:26,3,3,30,0.585050,0.132386,650


# Materialize Features to Online Store

In [6]:
store = FeatureStore(repo_path="specs")

start = datetime(2021, 4, 1)
end = datetime.utcnow()

try:
    store.materialize(start_date=start, end_date=end)
    print(f"✅ Successfully materialized features from {start} to {end}.")
except Exception as e:
    print("❌ Error during materialization:")
    print(str(e))

/opt/conda/lib/python3.11/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(
The `path` of the `RegistryConfig` starts with a plain `postgresql` string. We are updating this to `postgresql+psycopg` to ensure that the `psycopg3` driver is used by `sqlalchemy`. If you want to use `psycopg2` pass `postgresql+psycopg2` explicitely to `path`. To silence this warning, pass `postgresql+psycopg` explicitely to `path`.


Materializing 1 feature views from 2021-04-01 00:00:00+00:00 to 2025-06-06 09:41:46+00:00 into the postgres online store.

driver_hourly_stats2:


100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 216.65it/s]


✅ Successfully materialized features from 2021-04-01 00:00:00 to 2025-06-06 09:41:46.732083.


In [7]:
project_name = feature_store_file.get("project")
online_config = feature_store_file.get("online_store", {})

db_user = online_config.get("user")
db_password = online_config.get("password")
db_host = online_config.get("host")
db_port = online_config.get("port")
db_name = online_config.get("database")

if not all([db_user, db_password, db_host, db_port, db_name, project_name]):
    raise ValueError("Missing one or more required config values.")

conn_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(conn_str)

inspector = inspect(engine)
tables = inspector.get_table_names(schema="public")

expected_table = f"{project_name}_driver_hourly_stats2"
assert expected_table in tables, f"❌ Table '{expected_table}' not found in online store."

print(f"✅ Table '{expected_table}' exists in the online store.")

✅ Table 'feast_project_driver_hourly_stats2' exists in the online store.


# Retrieve Online Features

In [8]:
store = FeatureStore(repo_path="specs")  # adjust if needed

try:
    feature_vector = store.get_online_features(
        features=[
            "driver_hourly_stats2:conv_rate",
            "driver_hourly_stats2:acc_rate",
            "driver_hourly_stats2:avg_daily_trips",
        ],
        entity_rows=[
            {"driver_id": 1004},
            {"driver_id": 1005},
        ],
    ).to_dict()

    assert feature_vector and all(
        len(v) > 0 for v in feature_vector.values()
    ), "❌ No features returned from online store."

    print("✅ get_online_features returned non-empty data.")
    pprint(feature_vector)

except Exception as e:
    print("❌ Error calling get_online_features:")
    print(str(e))

/opt/conda/lib/python3.11/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(
The `path` of the `RegistryConfig` starts with a plain `postgresql` string. We are updating this to `postgresql+psycopg` to ensure that the `psycopg3` driver is used by `sqlalchemy`. If you want to use `psycopg2` pass `postgresql+psycopg2` explicitely to `path`. To silence this warning, pass `postgresql+psycopg` explicitely to `path`.


✅ get_online_features returned non-empty data.
{'acc_rate': [0.7248384356498718, 0.2648613750934601],
 'avg_daily_trips': [13, 533],
 'conv_rate': [0.9506867527961731, 0.8889309167861938],
 'driver_id': [1004, 1005]}
